In [ ]:
from qiskit_aer import AerSimulator
from qiskit.circuit.library import RealAmplitudes
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler, QiskitRuntimeService

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# If you did not previously save your credentials, use the following line instead:
service = QiskitRuntimeService(channel="ibm_quantum", token="***")

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import XGate, YGate, ZGate

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.draw()

┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

# Noise Model

Given a quantum circuit, this piece of code add noise gates, which are Pauli Operators depending on the wire-number of gate. The probablity for a random Pauli operator acting on the qubit after a one-qubit gate is $\alpha$ and probability of having a random Pauli operator acting on the qubit after a two-qubit gate is $\beta$.

error gate = [XGate( ), YGate( ), ZGate( )]

In [95]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import XGate, YGate, ZGate

def add_noise(circuit, alpha, beta):

    noisy_circuit = QuantumCircuit(circuit.num_qubits)

    # Lambda function to pick a random Pauli gate
    error_gate = [XGate(), YGate(), ZGate()]

    for instruction in circuit.data:
        gate = instruction.operation
        qubits = [circuit.find_bit(qubit).index for qubit in instruction.qubits]

        # Append the original gate to the new circuit
        noisy_circuit.append(gate, qubits)

        # Determine if noise should be added
        if gate.num_qubits == 1:
            # Add noise after a one-qubit gate with probability p1
            if np.random.rand() < alpha:
                pauli_gate = np.random.choice([0,1,2])
                noisy_circuit.append(error_gate[pauli_gate],[qubits[0]])

        elif gate.num_qubits == 2:
            # Add noise after a two-qubit gate with probability p2
            if np.random.rand() < beta:
                # Apply a random Pauli gate to each qubit in the two-qubit gate
                for qubit in qubits:
                    pauli_gate = np.random.choice([0,1,2])
                    noisy_circuit.append(error_gate[pauli_gate], [qubit])

    return noisy_circuit

# example of inclusion of noise in a bell circuit
noisy = add_noise(qc, 0.5, 0.5)
noisy.draw()


┌───┐┌───┐     ┌───┐
q_0: ┤ H ├┤ Z ├──■──┤ Z ├
     └───┘└───┘┌─┴─┐├───┤
q_1: ──────────┤ X ├┤ Z ├
               └───┘└───┘

# Transpile

Each quantum processor has a set of basis gates defined by the physical constraints of its hardware architecture. More complex quantum gates are then constructed from these basis gates using a process called transpilation or decomposition. The function **custom\_transpile** will take a circuit and transpile the gates into following basis gates;  {CX,ID,RZ,SX,X}

In [117]:
from qiskit import QuantumCircuit
import numpy as np

# Define a function to replace an H gate with basis gates
def decompose_h():
    # H gate decomposition using RZ, SX
    decomposed_circuit = QuantumCircuit(1)
    decomposed_circuit.rz(np.pi, 0)  # equivalent to Z gate in this context
    decomposed_circuit.sx(0)
    decomposed_circuit.rz(np.pi, 0)
    return decomposed_circuit

# Define a function to replace an RX gate with basis gates
def decompose_rx(theta):

    # RX(theta) decomposition using RZ and SX

    decomposed_circuit = QuantumCircuit(1)
    decomposed_circuit.rz(np.pi/2, 0)
    decomposed_circuit.sx(0)
    decomposed_circuit.rz(theta, 0)
    decomposed_circuit.sx(0)
    decomposed_circuit.rz(-np.pi/2, 0)
    return decomposed_circuit

# Define a function to replace an RY gate with basis gates
def decompose_ry(theta):

    # RY(theta) decomposition using RZ and SX

    decomposed_circuit = QuantumCircuit(1)
    decomposed_circuit.rz(np.pi, 0)
    decomposed_circuit.sx(0)
    decomposed_circuit.rz(theta, 0)
    decomposed_circuit.sx(0)
    return decomposed_circuit

def decompose_cp(theta):

    # CP(theta) = RZ(theta/2) -> CX -> RZ(-theta/2) -> CX -> RZ(theta/2)

    decomposed_circuit = QuantumCircuit(2)
    decomposed_circuit.rz(theta / 2, 1)
    decomposed_circuit.cx(0, 1)
    decomposed_circuit.rz(-theta / 2, 1)
    decomposed_circuit.cx(0, 1)
    decomposed_circuit.rz(theta / 2, 1)
    return decomposed_circuit

# Define a function to replace a SWAP gate with basis gates
def decompose_swap():

    # SWAP(a, b) = CX(a, b) -> CX(b, a) -> CX(a, b)

    decomposed_circuit = QuantumCircuit(2)  # 2 qubits for the swap
    decomposed_circuit.cx(0, 1)  # Apply CX(a, b)
    decomposed_circuit.cx(1, 0)  # Apply CX(b, a)
    decomposed_circuit.cx(0, 1)  # Apply CX(a, b)
    return decomposed_circuit


# Define a function to transpile a full circuit from scratch
def custom_transpile(circuit):
    transpiled_circuit = QuantumCircuit(circuit.num_qubits)

    # Update the loop to use operation, qubits, and clbits explicitly
    for instruction in circuit.data:
        instr = instruction.operation
        qargs = instruction.qubits
        gate_name = instr.name

        # Use find_bit to get the correct index for each qubit in qargs
        qubit_index = [circuit.find_bit(qubit).index for qubit in qargs]

        if gate_name == "h":
            # Decompose H gate
            decomposed_h = decompose_h()
            transpiled_circuit.compose(decomposed_h, qubit_index, inplace=True)

        elif gate_name == "rx":
            # Decompose RX gate
            theta = instr.params[0]
            decomposed_rx = decompose_rx(theta)
            transpiled_circuit.compose(decomposed_rx, qubit_index, inplace=True)

        elif gate_name == "ry":
            # Decompose RY gate
            theta = instr.params[0]
            decomposed_ry = decompose_ry(theta)
            transpiled_circuit.compose(decomposed_ry, qubit_index, inplace=True)

        elif gate_name == "cp":
            # Decompose CP gate
            theta = instr.params[0]
            decomposed_cp = decompose_cp(theta)
            transpiled_circuit.compose(decomposed_cp, qubit_index, inplace=True)

        elif gate_name == "swap":
            # Decompose SWAP gate
            decomposed_swap = decompose_swap()
            transpiled_circuit.compose(decomposed_swap, qubit_index, inplace=True)

        elif gate_name == "rz":
            # RZ is already a basis gate
            transpiled_circuit.rz(instr.params[0], qubit_index[0])

        elif gate_name == "sx":
            # SX is already a basis gate
            transpiled_circuit.sx(qubit_index[0])

        elif gate_name == "x":
            # X is already a basis gate
            transpiled_circuit.x(qubit_index[0])

        elif gate_name == "cx":
            # CX is already a basis gate
            transpiled_circuit.cx(qubit_index[0], qubit_index[1])

        # else:
        #     raise ValueError(f"Gate {gate_name} is not supported in the basis set.")

    return transpiled_circuit

# Example circuit with a mix of gates
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.rx(np.pi/3, 1)
qc.ry(np.pi/4, 0)

print("Original Circuit:")
print(qc)

# Transpile the circuit manually
transpiled_qc = custom_transpile(qc)

print("\nTranspiled Circuit:")
print(transpiled_qc)


Original Circuit:
     ┌───┐     ┌─────────┐
q_0: ┤ H ├──■──┤ Ry(π/4) ├
     └───┘┌─┴─┐├─────────┤
q_1: ─────┤ X ├┤ Rx(π/3) ├
          └───┘└─────────┘

Transpiled Circuit:
     ┌───────┐┌────┐┌───────┐      ┌───────┐ ┌────┐┌─────────┐┌────┐»
q_0: ┤ Rz(π) ├┤ √X ├┤ Rz(π) ├──■───┤ Rz(π) ├─┤ √X ├┤ Rz(π/4) ├┤ √X ├»
     └───────┘└────┘└───────┘┌─┴─┐┌┴───────┴┐├────┤├─────────┤├────┤»
q_1: ────────────────────────┤ X ├┤ Rz(π/2) ├┤ √X ├┤ Rz(π/3) ├┤ √X ├»
                             └───┘└─────────┘└────┘└─────────┘└────┘»
«                 
«q_0: ────────────
«     ┌──────────┐
«q_1: ┤ Rz(-π/2) ├
«     └──────────┘


# Draper Sum Algorithm

In [113]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator

def qft(circuit, n):
    """Apply the Quantum Fourier Transform to the first n qubits in the circuit."""
    for j in range(n):
        circuit.h(j)
        for k in range(j + 1, n):
            circuit.cp(np.pi / 2 ** (k - j), k, j)  # Controlled phase rotation

    # Reverse qubit order to complete QFT
    for i in range(n // 2):
        circuit.swap(i, n - i - 1)

def inverse_qft(circuit, n):
    """Apply the inverse Quantum Fourier Transform to the first n qubits in the circuit."""
    # Reverse the qubit order
    for i in range(n // 2):
        circuit.swap(i, n - i - 1)

    # Apply inverse QFT operations
    for j in range(n - 1, -1, -1):
        for k in range(j - 1, -1, -1):
            circuit.cp(-np.pi / 2 ** (j - k), j, k)  # Controlled phase rotation
        circuit.h(j)

def quantum_sum(a, b):
    """
    Quantum addition of two integers using the Draper adder algorithm.

    Parameters:
    - a (int): First integer to add.
    - b (int): Second integer to add.

    Returns:
    - QuantumCircuit: The circuit that adds a and b.
    """
    # Determine the number of qubits required
    n = max(a.bit_length(), b.bit_length()) + 1  # +1 to handle overflow

    # Create a quantum circuit with 2n qubits (n for each number)
    circuit = QuantumCircuit(2 * n)

    # Initialize the qubits to represent the binary forms of a and b
    # For number `a` in the first n qubits
    for i in range(n):
        if (a >> i) & 1:
            circuit.x(i)

    # For number `b` in the second n qubits
    for i in range(n):
        if (b >> i) & 1:
            circuit.x(i + n)

    # Apply QFT to the first register (where `a` is stored)
    qft(circuit, n)

    # Add the values of the second register (`b`) to the first register using controlled-phase gates
    for i in range(n):
        for j in range(i + 1):
            if (b >> j) & 1:
                circuit.cp(np.pi / 2 ** (i - j), j + n, i)

    # Apply inverse QFT to the first register to get the sum
    inverse_qft(circuit, n)

    # Measure the result from the first n qubits
    # circuit.measure_all()

    return circuit

In [100]:
# Example: Adding 1 + 2
a = 1
b = 2
adder_circuit = quantum_sum(a, b)

adder_circuit.measure_all()


aer_sim = AerSimulator()
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=3)
isa_qcl = pm.run(adder_circuit)

with Session(backend=aer_sim) as session:
    sampler = Sampler()
    result = sampler.run([isa_qcl]).result()

In [112]:
adder_circuit.draw()

┌───┐┌───┐                                              ┌───┐ ░ ┌─┐   »
    q_0: ┤ X ├┤ H ├─■─────────────X──■───────────────X──■────────┤ H ├─░─┤M├───»
         └───┘└───┘ │P(π/2) ┌───┐ │  │               │  │P(-π/2) ├───┤ ░ └╥┘┌─┐»
    q_1: ───────────■───────┤ H ├─X──┼──────■────────X──■────────┤ H ├─░──╫─┤M├»
                            └───┘    │      │                    └───┘ ░  ║ └╥┘»
    q_2: ────────────────────────────┼──────┼──────────────────────────░──╫──╫─»
         ┌───┐                       │P(π)  │P(π/2)                    ░  ║  ║ »
    q_3: ┤ X ├───────────────────────■──────■──────────────────────────░──╫──╫─»
         └───┘                                                         ░  ║  ║ »
    q_4: ──────────────────────────────────────────────────────────────░──╫──╫─»
                                                                       ░  ║  ║ »
 meas: 5/═════════════════════════════════════════════════════════════════╩══╩═»
                                                                          0  1 »
meas8: 5/══════════════════════════════════════════════════════════════════════»
                                                                               »
«                   ░ ┌─┐            
«    q_0: ──────────░─┤M├────────────
«                   ░ └╥┘┌─┐         
«    q_1: ──────────░──╫─┤M├─────────
«         ┌─┐       ░  ║ └╥┘┌─┐      
«    q_2: ┤M├───────░──╫──╫─┤M├──────
«         └╥┘┌─┐    ░  ║  ║ └╥┘┌─┐   
«    q_3: ─╫─┤M├────░──╫──╫──╫─┤M├───
«          ║ └╥┘┌─┐ ░  ║  ║  ║ └╥┘┌─┐
«    q_4: ─╫──╫─┤M├─░──╫──╫──╫──╫─┤M├
«          ║  ║ └╥┘ ░  ║  ║  ║  ║ └╥┘
« meas: 5/═╩══╩══╩═════╬══╬══╬══╬══╬═
«          2  3  4     ║  ║  ║  ║  ║ 
«meas8: 5/═════════════╩══╩══╩══╩══╩═
«                      0  1  2  3  4

Following the left hand convention, the count for 01011 is highest, showing the result of the operation to be 3(the last two digit of final state)

In [102]:
pub_result1 = result[0]
print(f"Counts for the meas output register: {pub_result1.data.meas.get_counts()}")

Counts for the meas output register: {'01011': 632, '01001': 113, '01000': 133, '01010': 146}


# Merging

Now we merge the **noise model** with **quantum_sum** circuit after the **transpilation** into the basis gate set.

In [114]:
# Transpile, Add Noise, and Simulate with Different Noise Levels
def simulate_quantum_sum_with_noise(a, b, p1, p2):
    qc = quantum_sum(a, b)
    transpiled_circuit = custom_transpile(qc)
    final_noisy_qc = add_noise(transpiled_circuit, p1, p2)

    return final_noisy_qc


In [125]:
my_circ1 = simulate_quantum_sum_with_noise(1, 2, 0.1, 0.1)
my_circ1.measure_all()

aer_sim = AerSimulator()
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=3)
isa_qcl = pm.run(my_circ1)

with Session(backend=aer_sim) as session:
    sampler = Sampler()
    result = sampler.run([isa_qcl]).result()

pub_result1 = result[0]
print(f"Counts for the meas output register: {pub_result1.data.meas.get_counts()}")

Counts for the meas output register: {'010001': 47, '010100': 556, '010110': 136, '010111': 111, '010000': 7, '010010': 122, '010101': 45}


In [129]:
my_circ2 = simulate_quantum_sum_with_noise(1, 2, 0.2, 0.2)
my_circ2.measure_all()

aer_sim = AerSimulator()
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=3)
isa_qcl = pm.run(my_circ2)

with Session(backend=aer_sim) as session:
    sampler = Sampler()
    result = sampler.run([isa_qcl]).result()

pub_result1 = result[0]
print(f"Counts for the meas output register: {pub_result1.data.meas.get_counts()}")

Counts for the meas output register: {'000111': 586, '000010': 56, '000101': 115, '000110': 56, '000100': 103, '000001': 92, '000011': 15, '000000': 1}


In [133]:
my_circ3 = simulate_quantum_sum_with_noise(1, 2, 0.3, 0.3)
my_circ3.measure_all()

aer_sim = AerSimulator()
pm = generate_preset_pass_manager(backend=aer_sim, optimization_level=3)
isa_qcl = pm.run(my_circ3)

with Session(backend=aer_sim) as session:
    sampler = Sampler()
    result = sampler.run([isa_qcl]).result()

pub_result1 = result[0]
print(f"Counts for the meas output register: {pub_result1.data.meas.get_counts()}")

Counts for the meas output register: {'010110': 47, '010000': 455, '010100': 39, '010011': 143, '010001': 182, '010101': 12, '010111': 63, '010010': 83}


In [116]:
# A general noisy quantum circuit

simulate_quantum_sum_with_noise(1, 2, 0.3, 0.3).draw()

┌───┐┌───────┐┌────┐┌───┐┌───────┐┌─────────┐┌───┐┌──────────┐┌───┐»
q_0: ┤ X ├┤ Rz(π) ├┤ √X ├┤ Z ├┤ Rz(π) ├┤ Rz(π/4) ├┤ X ├┤ Rz(-π/4) ├┤ X ├»
     └───┘└───────┘└────┘└───┘└───────┘└─────────┘└─┬─┘└──────────┘└─┬─┘»
q_1: ───────────────────────────────────────────────■────────────────■──»
                                                                        »
q_2: ───────────────────────────────────────────────────────────────────»
                                                                        »
q_3: ───────────────────────────────────────────────────────────────────»
     ┌───┐                                                              »
q_4: ┤ X ├──────────────────────────────────────────────────────────────»
     └───┘                                                              »
q_5: ───────────────────────────────────────────────────────────────────»
                                                                        »
«     ┌─────────┐┌─────────┐  ┌───┐  ┌───┐┌──────────┐           ┌───┐»
«q_0: ┤ Rz(π/4) ├┤ Rz(π/8) ├──┤ X ├──┤ X ├┤ Rz(-π/8) ├───────────┤ X ├»
«     └┬───────┬┘└──┬────┬─┘┌─┴───┴─┐└─┬─┘└──┬───┬───┘┌─────────┐└─┬─┘»
«q_1: ─┤ Rz(π) ├────┤ √X ├──┤ Rz(π) ├──┼─────┤ X ├────┤ Rz(π/4) ├──┼──»
«      └───────┘    └────┘  └───────┘  │     └───┘    └─────────┘  │  »
«q_2: ─────────────────────────────────■───────────────────────────■──»
«                                                                     »
«q_3: ────────────────────────────────────────────────────────────────»
«                                                                     »
«q_4: ────────────────────────────────────────────────────────────────»
«                                                                     »
«q_5: ────────────────────────────────────────────────────────────────»
«                                                                     »
«     ┌─────────┐                                                          »
«q_0: ┤ Rz(π/8) ├──────────────────────────────────────────────────────────»
«     └──┬───┬──┘┌───┐┌──────────┐┌───┐┌─────────┐┌─────────┐  ┌───┐       »
«q_1: ───┤ X ├───┤ Z ├┤ Rz(-π/4) ├┤ X ├┤ Rz(π/4) ├┤ Rz(π/2) ├──┤ X ├───────»
«        └─┬─┘   ├───┤└──────────┘└─┬─┘└┬───────┬┘└──┬────┬─┘┌─┴───┴─┐┌───┐»
«q_2: ─────■─────┤ X ├──────────────■───┤ Rz(π) ├────┤ √X ├──┤ Rz(π) ├┤ Z ├»
«                └───┘                  └───────┘    └────┘  └───────┘└───┘»
«q_3: ─────────────────────────────────────────────────────────────────────»
«                                                                          »
«q_4: ─────────────────────────────────────────────────────────────────────»
«                                                                          »
«q_5: ─────────────────────────────────────────────────────────────────────»
«                                                                          »
«                  ┌───┐         ┌───┐                                      »
«q_0: ───────■─────┤ Y ├─────────┤ X ├─────────────■────────────────────────»
«     ┌───┐  │  ┌──┴───┴───┐┌───┐└─┬─┘┌─────────┐  │  ┌──────────┐          »
«q_1: ┤ X ├──┼──┤ Rz(-π/2) ├┤ X ├──┼──┤ Rz(π/2) ├──┼──┤ Rz(-π/4) ├──────────»
«     └─┬─┘┌─┴─┐└──┬───┬───┘└─┬─┘  │  └─────────┘┌─┴─┐├─────────┬┘┌───┐┌───┐»
«q_2: ──┼──┤ X ├───┤ Y ├──────┼────■─────────────┤ X ├┤ Rz(π/4) ├─┤ X ├┤ X ├»
«       │  └───┘   └───┘      │                  └───┘└─────────┘ └─┬─┘└───┘»
«q_3: ──┼─────────────────────┼─────────────────────────────────────┼───────»
«       │                     │                                     │  ┌───┐»
«q_4: ──■─────────────────────■─────────────────────────────────────■──┤ X ├»
«                                                                      └───┘»
«q_5: ──────────────────────────────────────────────────────────────────────»
«                                                                           »
«                                           ┌───┐     ┌──────────┐     »
«q_0: ─────────────

Below is the transpilation of a bell circuit showing the breakdown of Hadamard and CNOT gate in the basis gate set.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# If you did not previously save your credentials, use the following line instead:
service = QiskitRuntimeService(channel="ibm_quantum", token="2e1b3eb3e672ac4571fddd2dc631f8876fd09fb538c9179a2e054748696c04d27851088e6ec5300cbff17e357a0f9235d0cbe16db1966947e2687c5dd4cd80dc")
# service = QiskitRuntimeService()

backend = service.least_busy(simulator=False, operational=True)

# Convert to an ISA circuit and layout-mapped observables.
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

isa_circuit.draw(idle_wires=False)

global phase: π/2
          ┌────────┐ ┌────┐          ┌──────┐ ┌────────┐ ┌────┐┌─────────┐»
q_0 -> 0 ─┤ Rz(-π) ├─┤ √X ├──────────┤1     ├─┤ Rz(-π) ├─┤ √X ├┤ Rz(π/4) ├»
         ┌┴────────┴┐├────┤┌────────┐│  Ecr │┌┴────────┴┐├────┤├─────────┤»
q_1 -> 1 ┤ Rz(-π/2) ├┤ √X ├┤ Rz(-π) ├┤0     ├┤ Rz(5π/6) ├┤ √X ├┤ Rz(π/2) ├»
         └──────────┘└────┘└────────┘└──────┘└──────────┘└────┘└─────────┘»
«         ┌────┐
«q_0 -> 0 ┤ √X ├
«         └────┘
«q_1 -> 1 ──────
«

Below is the estimation of quantum_sum(1,1) without any noise and Custom transpilation, on the utility-scale QPU of IBM. This is discussed in section 3 of the write-up file.

In [134]:
from qiskit.quantum_info import SparsePauliOp

observables_labels = ["ZIII","IZII","IIZI","IIIZ","XIII","IXII","IIXI","IIIX","ZZII","IZZI","IIZZ","ZIIZ","XXII","IXXI","IIXX","XIIX"]
observables = [SparsePauliOp(label) for label in observables_labels]

In [ ]:
qc = quantum_sum(1,2)

backend = service.least_busy(simulator=False, operational=True)

# Convert to an ISA circuit and layout-mapped observables.
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

isa_circuit.draw('mpl', idle_wires=False)

In [ ]:
# Construct the Estimator instance.
from qiskit_ibm_runtime import EstimatorV2 as Estimator

estimator = Estimator(mode=backend)
estimator.options.resilience_level = 1
estimator.options.default_shots = 1000

mapped_observables = [observable.apply_layout(isa_circuit.layout) for observable in observables]

# One pub, with one circuit to run against five different observables.
job = estimator.run([(isa_circuit, mapped_observables)])

# Use the job ID to retrieve your job data later
print(f">>> Job ID: {job.job_id()}")

>>> Job ID: cwfz4zk40e000088mfm0


In [ ]:
pub_result = job.result()[0]

job_result = job.result()

for idx, pub_result in enumerate(job_result):
    print(f"Expectation values for pub {idx}: {pub_result.data.evs}")

Expectation values for pub 0: [ 1.00824742 -1.02444208 -0.49897331 -0.59521332 -0.00618557  0.03613177
 -0.10061602 -0.07284079 -1.03928171  0.50719823  0.56612022 -0.59185919
 -0.02244669 -0.00442968  0.01311475  0.04840484]


THE FIRST TWO ELEMENT OF THE PUB SHOWS THE STATE |01>, WHICH IS 2 IN BINARY(RIGHT-HAND CONVENTION)